In [ ]:
print('Gajraj')

In [ ]:
!pip install duckdb
!pip install langchain openai
!pip install unstructured chromadb
!pip install tiktoken

In [1]:
import pandas as pd
path = '/workspaces/codesandbox-template-blank/Code_for_Embeddings'
account_vulnerabilities_df=pd.read_csv(f'{path}/data/account_vulnerabilities.csv')
public_tools_df=pd.read_csv(f'{path}/data/public_tools.csv')
versions_data_df=pd.read_csv(f'{path}/data/versions_data.csv')
vulnerabilities_df=pd.read_csv(f'{path}/data/vulnerabilities.csv')
decisions_data_df=pd.read_csv(f'{path}/data/decisions_data.csv')

In [2]:
import duckdb
duckdb.default_connection.execute("SET GLOBAL pandas_analyze_sample=100000")
def sql_helper(sql):
    return duckdb.query(sql).to_df()

def sql_helper_one(sql):
    return duckdb.query(sql).fetchone()

In [3]:
sql=f"""
select *
from versions_data_df
limit 10
"""
df=sql_helper(sql)
# df.columns
df

,version_id,number,tool_id,tool_name,major_version_number,trusted,released_at,commit_url,source_url,release_source,release_id,detected in # of stacks,release codename,lts,end of life at,end of active support at,release at,number.1
0,3,7.9.0,2739,Babel,7,False,NaN,NaN,NaN,NaN,NaN,20,NaN,NaN,NaN,NaN,NaN,NaN
1,4,1.19.1,7035,Prettier,1,True,2019-11-09 12:21:15 UTC,NaN,https://libraries.io/npm/prettier/1.19.1,librariesio,NaN,113,NaN,NaN,NaN,NaN,NaN,NaN
2,5,4.17.15,2438,Lodash,4,True,2019-07-19 02:28:46 UTC,NaN,https://libraries.io/npm/lodash/4.17.15,librariesio,NaN,87,NaN,NaN,NaN,NaN,NaN,NaN
3,6,4.2.0,6112,Ant Design,4,True,2020-04-29 15:04:27 UTC,https://github.com/ant-design/ant-design/commi...,NaN,github,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
4,7,4.4.3,6749,styled-components,4,False,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
5,8,5.1.5,3350,React Router,5,False,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN
6,9,13.13.5,1011,Node.js,13,False,NaN,NaN,NaN,NaN,618.0,0,NaN,False,2020-06-01 00:00:00 UTC,2020-04-01 00:00:00 UTC,2019-10-22 00:00:00 UTC,13.0
7,10,16.13.1,1020,React,16,True,2020-03-19 20:29:38 UTC,https://github.com/facebook/react/commit/da834...,https://github.com/facebook/react/releases/tag...,github,666.0,114,NaN,False,NaN,2020-10-20 00:00:00 UTC,2017-09-26 00:00:00 UTC,16.0
8,11,4.42.0,1682,Webpack,4,True,2020-03-02 08:42:20 UTC,NaN,https://libraries.io/npm/webpack/4.42.0,librariesio,NaN,13,NaN,NaN,NaN,NaN,NaN,NaN
9,12,12.0.1,5446,Stylelint,12,True,2019-12-25 13:17:13 UTC,NaN,https://libraries.io/npm/stylelint/12.0.1,librariesio,NaN,1,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
vulnerabilities_df.columns
#Need to ask chetan for version range
#Issues and bugs detected
#EOL
#LTS
#VERSION-DIFFERNCE
#SCORE
#IMPROVE SUGGESTION WITH SCORE
# APP
    # 1. TOOL-VERSION-SCORE -> SUGGESTION ONLY FOR VERSION
    # APP->REPO->TOOLS

In [ ]:
# account_vulnerabilities_df.columns
# public_tools_df.columns
# account_vulnerabilities_df
versions_data_df.columns

In [16]:
id=3350
sql = f"""
    SELECT major_version_number
    FROM versions_data_df
    where tool_id={id}
    order by number
"""
df = sql_helper(sql)
min_version = df.major_version_number.min()
max_version = df.major_version_number.max()
sql = f"""
    select  
        sum("detected in # of stacks") as total_no_of_stack
    from versions_data_df
    where tool_id={id}
"""
total_stack_for_this_tools=sql_helper_one(sql)[0]
sql = f"""
    SELECT 
        version_id,number,major_version_number,released_at,trusted,"detected in # of stacks" as no_of_stack,        
        ({max_version} - major_version_number) as major_version_difference,
        CASE WHEN major_version_difference>1 THEN -2 WHEN major_version_difference=1 THEN -1 ELSE 0 END AS change_level,
        ((no_of_stack/{total_stack_for_this_tools})*100) as percentage,
        0 as cosine_distance
    FROM versions_data_df
    where tool_id={id}
    order by number
"""
masterdb=duckdb.query(sql).to_df()


# ->number[Exist in DB]
# ->major_version_number[Exist in DB]
# ->Weightage
# ->trusted[Exist in DB]
# ->released_at[Exist in DB]
# ->detected in # of stacks[Exist in DB]
# ->Change-level[High,Medium,Low][Need to Generate]
# ->Percentage of stack[against total # of stack][Need to Generate]
# ->Structured changelog[Documents][Need to Generate]
# ->Target_Version[Suggestions][Need to Generate]

In [23]:
sql=f"""
    select *
    from masterdb
    order by number
"""
sql_helper(sql)

,version_id,number,major_version_number,released_at,trusted,no_of_stack,major_version_difference,change_level,percentage,cosine_distance
0,2306,1.2.1,1,NaN,False,0,4,-2,0.000000,0
1,8106,2.8.1,2,2016-09-13 16:58:03 UTC,True,1,3,-2,0.549451,0
2,7904,3.2.0,3,2017-09-26 17:24:50 UTC,True,1,2,-2,0.549451,0
3,315,3.2.1,3,2018-03-07 18:27:07 UTC,True,1,2,-2,0.549451,0
4,8287,3.2.4,3,2019-08-23 17:28:44 UTC,True,0,2,-2,0.000000,0
5,1522,4.1.2,4,NaN,False,0,1,-1,0.000000,0
6,1497,4.2.0,4,2017-08-24 01:26:13 UTC,True,9,1,-1,4.945055,0
7,1185,4.2.2,4,2017-08-25 00:11:08 UTC,True,6,1,-1,3.296703,0
8,1910,4.2.7,4,NaN,False,0,1,-1,0.000000,0
9,110,4.3.1,4,2018-06-06 19:30:45 UTC,True,28,1,-1,15.384615,0


In [ ]:
def convert_to_list(i):
    items=[]
    for item in i.replace('"','').replace("[",'').replace("]",'').split(","):
        items.append(item.strip())
    return items

def convert_to_documents(version_id, number, major_version_number, released_at,trusted, no_of_stack, major_version_difference, change_level,percentage):
    document=f"""
        Description:
            This document represent information about recommendation of solution based on user inputted query.
            Where user need an answer in form of recommandation for perticular scenario.
            It's following information
            version_id: {version_id}
            number: {number}
            major_version_number: {major_version_number}
            released_at: {released_at}
            trusted: {trusted}
            no_of_stack: {no_of_stack}
            major_version_difference: {major_version_difference}
            change_level: {change_level}
            percentage: {percentage}
            It'll help to recognize recommendation
    """
    return document

In [ ]:
from langchain.document_loaders import DirectoryLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

In [ ]:
# from langchain.embeddings import OpenAIEmbeddings
# os.environ["OPENAI_API_TYPE"] = "azure"
# os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
# os.environ["OPENAI_API_BASE"] = "https://stackgptembeddings.openai.azure.com/"
# os.environ["OPENAI_API_KEY"] = "af7c74f224e9457b9532f47e8d64a7ae"

# embeddings = OpenAIEmbeddings(deployment="stackgptembeddings")
# embeddings = OpenAIEmbeddings()

In [ ]:
type(embeddings)

In [ ]:
# Import Document class
from langchain.docstore.document import Document

document_ids=[]
original_docs=[]
for inx,item in masterdb.iterrows():
    # print(item)
    initial_content = convert_to_documents(item.version_id,item.number,item.major_version_number,item.released_at,item.trusted,item.no_of_stack,item.major_version_difference,item.change_level,item.percentage)
    document_id = str(item.version_id)
    # Create an instance of Document with initial content and metadata
    original_doc = Document(page_content=initial_content, metadata={"version_id":item.version_id,"number":item.number,"major_version_number":item.major_version_number,"released_at":item.released_at,"trusted":item.trusted,"no_of_stack":item.no_of_stack,"major_version_difference":item.major_version_difference,"change_level":item.change_level,"percentage":item.percentage})
    document_ids.append(document_id)
    original_docs.append(original_doc)
    # Initialize a Chroma instance with the original document

In [ ]:
search_db = Chroma.from_documents(
    collection_name="version_collection",
    documents=original_docs,
    embedding=embeddings,
    ids=document_ids,
    persist_directory='/workspaces/codesandbox-template-blank/Code_for_Embeddings/db_recommendation'
)

In [ ]:
search_db.persist()

In [ ]:
search_db=None

In [ ]:

persist_directory = '/workspaces/codesandbox-template-blank/Code_for_Embeddings/db_recommendation'
vectordb = Chroma(persist_directory=persist_directory, embedding_function=embeddings)

In [ ]:
vectordb.similarity_search('recommandation for perticular scenario')

In [ ]:
df=pd.read_parquet("./db_recommendation/chroma-embeddings.parquet")